In [ ]:
!nvidia-smi

In [ ]:
!pip3 install pytorch2keras
!pip install scipy
!pip install imgaug==0.2.6
!pip install -q keras
!pip install -q tqdm
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!pip install -q tensorflow-model-optimization
!apt-get -qq install xxd

# Imports

In [ ]:
# PYTORCH
import torch.nn.functional as F
import torch
import torch.nn.utils.prune as prune
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch.nn as nn
from torchvision import transforms, datasets, models
from torchsummary import summary
from torch.autograd import Variable


# KERAS
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Model
# force a channel ordering
from keras import backend
# force channels-first ordering
backend.set_image_data_format('channels_first')
import tensorflow as tf
tf.executing_eagerly()
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from keras import backend as K
from tensorflow.keras import layers, models
import tensorflow as tf


# GERAL
from pytorch2keras.converter import pytorch_to_keras
from __future__ import print_function
from __future__ import division
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import numpy as np
from tqdm import tqdm
import random
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sys

# Preparação dos dados

O dataset deve estar em formato zip no google drive

In [ ]:
# monta o google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/chest_xray.zip

In [ ]:
!rm -rf /content/chest_xray

# Funções

In [ ]:
def print_size_of__torch_model(model):
  torch.save(model.state_dict(), "temp.p")
  print('Size (MB):', os.path.getsize("temp.p")/1e6)
  os.remove('temp.p')

def print_size_of_keras_model(model):
  model.save("temp.p")
  print('Size (MB):', os.path.getsize("temp.p")/1e6)
  os.remove('temp.p')

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def prepararDados(input_size):
  datadir = '/content/chest_xray'
  traindir = datadir + '/train/'
  validdir = datadir + '/val/'
  testdir = datadir + '/test/'
 
  batch_size = 32

  image_transforms = {
    'train': transforms.Compose([    
        transforms.Resize((input_size,input_size)),        
        transforms.RandomHorizontalFlip(), 
        transforms.RandomResizedCrop((input_size,input_size)),  
        transforms.ToTensor(), 
      ]),

    'test': transforms.Compose([
        transforms.Resize((input_size,input_size)),      
        transforms.ToTensor(),
            ]),

    'val': transforms.Compose([
        transforms.Resize((input_size,input_size)),   
        transforms.ToTensor(),
            ])
  }
  
  # Datasets
  data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root=testdir, transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root=testdir, transform=image_transforms['test'])
  } 


  dataloaders = {
      'train': DataLoader(data['train'], shuffle=True, batch_size=batch_size, num_workers=0),
      'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
      'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
  }
  return dataloaders


def preparaDadosKeras(input_size=224, bs=32):
  train_datagen = ImageDataGenerator(horizontal_flip=True, data_format="channels_first", rescale=1/255)
  
  test_datagen = ImageDataGenerator(data_format="channels_first", rescale=1/255)

  train_datagen = train_datagen.flow_from_directory('/content/chest_xray/train', target_size=(input_size, input_size), shuffle=True, interpolation='bilinear', batch_size=bs, class_mode='sparse')

  test_datagen = test_datagen.flow_from_directory('/content/chest_xray/test', target_size=(input_size, input_size), shuffle=True, interpolation='bilinear', batch_size=bs, class_mode='sparse')
  
  data_generators = {
      'train': train_datagen,
      'val': test_datagen,
      'test': test_datagen
  }
  return data_generators
        
def carregar_resnet():
  resnet, input_size = initialize_model('resnet', 2, True, use_pretrained=True)
  resnet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Resnet18/resnet.acc'))
  return resnet, input_size

def carregar_densenet():
  densenet, input_size = initialize_model('densenet', 2, True, use_pretrained=True)
  densenet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Densenet121/densenet.acc'))
  return densenet, input_size

def carregar_alexnet():
  alexnet, input_size = initialize_model('alexnet', 2, True, use_pretrained=True)
  alexnet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/AlexNet/alexnet.acc'))
  return alexnet, input_size

def carregar_googlenet():
  googlenet, input_size = initialize_model('googlenet', 2, True, use_pretrained=True)
  googlenet.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/GoogLeNet/googlenet.acc'))
  return googlenet, input_size

def carregar_inception():
  inception, input_size = initialize_model('inception', 2, True, use_pretrained=True)
  inception.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Inception V3/inception.acc'))
  return inception, input_size

def carregar_vgg():
  vgg = models.vgg19_bn(pretrained=True)
  num_ftrs = vgg.classifier[6].in_features
  vgg.classifier[6] = nn.Linear(num_ftrs, 2)
  vgg.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/VGG/vgg.acc'))
  return vgg, 224

def carregar_cnn(model_save_name):
    model = Network()    
    model.load_state_dict(torch.load(model_save_name))
    return model

def carregar_ensemble():
  #load the models
  resnet, input_size = carregar_resnet()
  vgg, input_size = carregar_vgg()
  densenet, input_size = carregar_densenet()
  alexnet, input_size = carregar_alexnet()
  googlenet, input_size = carregar_googlenet()
  ensemble = {'resnet': resnet, 'vgg': vgg, 'densenet': densenet, 'alexnet': alexnet, 'googlenet': googlenet}
  return ensemble

def set_parameter_requires_grad(model, feature_extracting):
  if feature_extracting:
    for param in model.parameters():
      param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
  # Initialize these variables which will be set in this if statement. Each of these
  
  #   variables is model specific.
  model_ft = None
  input_size = 0

  if model_name == "resnet":
    """ Resnet18
    """
    model_ft = models.resnet18(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    set_parameter_requires_grad(model_ft.layer4, True)
    
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

  elif model_name == "alexnet":
    """ Alexnet
    """
    model_ft = models.alexnet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft.features, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224

  elif model_name == "vgg":
    """ VGG11_bn
    """
    model_ft = models.vgg19_bn(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224

  elif model_name == "squeezenet":
    """ Squeezenet
    """
    model_ft = models.squeezenet1_0(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft.features, feature_extract)
    model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
    model_ft.num_classes = num_classes
    input_size = 224

  elif model_name == "densenet":
    """ Densenet
    """
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    input_size = 224

  elif model_name == "inception":
    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    model_ft = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 299

  elif model_name == "googlenet":
    """ GoogLeNet
    """
    model_ft = models.googlenet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

  else:
      print("Invalid model name, exiting...")
      exit()

  return model_ft, input_size

def testePruning(model, data_loader):
  if (torch.cuda.is_available()):
    model = model.cuda()
  model.eval()
  running_loss = 0.0
  running_correct = 0
  tot_predictions = []
  labels = []
  for batch in data_loader:
    data , target = batch
    if torch.cuda.is_available():
      data,target = data.cuda(),target.cuda()    
    output = model(data)
    loss = F.cross_entropy(output,target)
    running_loss += loss.item()
    preds = output.data.max(dim=1,keepdim=True)[1]
    running_correct += preds.eq(target.data.view_as(preds)).cpu().sum().item()
    tot_predictions.extend(preds.data.cpu().numpy())
    labels.extend(target.data.cpu().numpy())
  loss = running_loss/len(data_loader.dataset)
  accuracy = 100. * running_correct/len(data_loader.dataset)  
  return accuracy, loss

def carrega_model(name):
  model = None
  if name == 'Resnet18':
    model, input_size = carregar_resnet()
  elif name == 'VGG':
    model, input_size = carregar_vgg()
  elif name == 'Densenet121': 
    model, input_size = carregar_densenet()
  elif name == 'AlexNet': 
    model, input_size = carregar_alexnet()
  elif name == 'GoogLeNet':
    model, input_size = carregar_googlenet()
  elif name == 'Aluno':
    model = carregar_cnn('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno_kd_v2.acc')
  return model 

def calcSTD(model_name):
  model = carrega_model(model_name) 
  
  #calc Media
  media = 0.0
  n = 0
  # For each layer except the output one
  for layer in model.modules(): 
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
      array = layer.weight.detach().numpy()
      media += np.mean(array)
      n += 1
  media = (media/n)

  # Calc std
  n = 0
  std = 0.0
  sum = 0 
  for layer in model.modules(): 
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
      array = layer.weight.detach().numpy()
      n += array.size
      sub = array - media
      pot = sub**2
      sum += np.sum(pot)
  sum = sum/(n-1)
  std = sum**0.5
  return std
  
def carrega_model_prunned(name):
  model = None
  if name == 'Resnet18':
    model, input_size = initialize_model('resnet', 2, True, use_pretrained=True)
    model.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Resnet18/resnet18-prunned.acc'))
  elif name == 'VGG':
    model = models.vgg19_bn(pretrained=True)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, 2)
    model.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/VGG/vgg-prunned.acc'))
  elif name == 'Densenet121': 
    model, input_size = initialize_model('densenet', 2, True, use_pretrained=True)
    model.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/Densenet121/densenet121-prunned.acc'))
  elif name == 'AlexNet': 
    model, input_size = initialize_model('alexnet', 2, True, use_pretrained=True)
    model.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/AlexNet/alexnet-prunned.acc'))
  elif name == 'GoogLeNet':
    model, input_size = initialize_model('googlenet', 2, True, use_pretrained=True)
    model.load_state_dict(torch.load('/content/drive/My Drive/Modelos Salvos/GoogLeNet/googlenet-prunned.acc'))
  elif name == 'Aluno':
    model = carregar_cnn('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned.acc')
  return model 

def carrega_model_prunned_keras(name):
  model = None
  if name == 'Resnet18':
    model = keras.models.load_model('/content/drive/My Drive/Modelos Salvos/Resnet18/resnet18-prunned-keras.h5')
  elif name == 'AlexNet': 
    model = keras.models.load_model('/content/drive/My Drive/Modelos Salvos/AlexNet/alexnet-prunned-keras.h5')
  elif name == 'Aluno':
    model = keras.models.load_model('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.h5')
  return model 


def convert_model(input_tensor, model_to_transfer):
    
    # define input tensor
    input_var = Variable(torch.FloatTensor(input_tensor))

    # get PyTorch model
    model_to_transfer.eval()

    # convert PyTorch model to Keras
    model = pytorch_to_keras(
        model_to_transfer,
        input_var,
        [input_var.shape[-3:]],
        change_ordering=True,
        verbose=False,
        name_policy="keep",
    )

    return model



def testeDoModelo(model, data_loader):
  if (torch.cuda.is_available()):
    model = model.cuda()
  model.eval()
  running_loss = 0.0
  running_correct = 0
  tot_predictions = []
  labels = []
  for batch in data_loader:
    data , target = batch
    if torch.cuda.is_available():
      data,target = data.cuda(),target.cuda()
    
    output = model(data)
    loss = F.cross_entropy(output,target)
    running_loss += loss.item()
    preds = output.data.max(dim=1,keepdim=True)[1]
    running_correct += preds.eq(target.data.view_as(preds)).cpu().sum().item()
    
    tot_predictions.extend(preds.data.cpu().numpy())
    labels.extend(target.data.cpu().numpy())
  loss = running_loss/len(data_loader.dataset)
  accuracy = 100. * running_correct/len(data_loader.dataset)
  print(f'Loss is {loss:{5}.{2}} Accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{6}.{4}}%\n')
  return tot_predictions, labels

def metricasModelo(model, data_loader):
  if (torch.cuda.is_available()):
    model = model.cuda()
    
  print('========= Métricas do modelo ============\n')
  y_pred, y_test = testeDoModelo(model, data_loader)
  target_names = ['normal', 'pneumonia']
  print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
  print('\n========= Confusion Matrix ============\n')
  confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
  fig, ax = plt.subplots(figsize=(3,3))
  ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
  for i in range(confmat.shape[0]):
      for j in range(confmat.shape[1]):
          ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
  plt.xlabel('predicted label')
  plt.ylabel('true label')
  plt.tight_layout()
  plt.savefig('confusion_matrix.png', dpi=300)
  plt.show()


In [ ]:
data_generators = preparaDadosKeras(224)
print('Keras')
print(data_generators['train'].class_indices)

print('Pytorch')
loaders = prepararDados(224)
print(loaders['train'].dataset.class_to_idx)

# Modelo Estudante


In [ ]:
class Network(nn.Module):
    def __init__(self):
        """CNN Builder."""
        super().__init__()

        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=24, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
           
        )
        
        self.fc_layer = nn.Sequential(
           
            nn.Linear(14*14*24, 2),
       
         
        )

    def forward(self, x):  

        # conv layers
        x = self.conv_layer(x)       

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x



In [ ]:
summary(Network().cuda(), (3, 224, 224))

KERAS


In [ ]:
def get_model():
    img_size = (224,224)
    inputs = keras.Input(shape=(3,224,224))   

    x = layers.Conv2D(16, 3, strides=1, padding="same")(inputs)
    x = layers.BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(x)
    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

    x = layers.Conv2D(32, 3, strides=1, padding="same")(x)
    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)    

    x = layers.Conv2D(32, 3, strides=1, padding="same")(x)
    x = layers.BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(x)
    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)

    x = layers.Conv2D(24, 3, strides=1, padding="same")(x)

    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)
    
    x = layers.Flatten(data_format='channels_last')(x)

    x = layers.Dense(2)(x)

    # Define the model
    model = keras.Model(inputs, x)
    return model

In [ ]:
def get_converted_model():
  pytorch_model = carrega_model_prunned('Aluno')
  pytorch_model.eval()
  params = {}
  conv_idx = 0
  bn_idx = 0
  for module in pytorch_model.modules():
    if isinstance(module, nn.Conv2d):     
      params['conv'+str(conv_idx)] = [module.weight.data.numpy().transpose(2,3,1, 0), module.bias.data.numpy()]
      conv_idx += 1

    if isinstance(module, nn.BatchNorm2d):     
      params['bn'+str(bn_idx)] = [module.weight.data.numpy(), module.bias.data.numpy(), module.running_mean.data.numpy(), module.running_var.data.numpy()]
      bn_idx += 1      

    if isinstance(module, nn.Linear):
      params['linear'] = [module.weight.data.numpy().transpose(1,0), module.bias.data.numpy()]
  keras_model = get_model()
  conv_idx = 0
  bn_idx = 0
  for l in keras_model.layers: 
    if isinstance(l, layers.Conv2D):
      l.set_weights(np.array(params['conv'+str(conv_idx)]))
      conv_idx += 1

    if isinstance(l, layers.Dense):
      l.set_weights(np.array(params['linear']))

    if isinstance(l, layers.BatchNormalization):
      l.set_weights(np.array(params['bn'+str(bn_idx)]))
      bn_idx += 1
      
  return keras_model

get_converted_model().summary()


In [ ]:
loaders = prepararDados(224)
generators = preparaDadosKeras(224)

pytorch_model = carrega_model_prunned('Aluno')
pytorch_model = pytorch_model.eval()

keras_model = get_converted_model()
keras_model.save('/content/drive/My Drive/Modelos Salvos/Aluno/aluno-prunned-keras.h5')

keras_corrects = 0
pytorch_corrects = 0
total_images = 0

for batch in tqdm(loaders['test']):
  images, labels =  batch
  labels = labels.data.numpy()

  keras_input = images.data.numpy()
  keras_output = keras_model.predict(keras_input)
  keras_preds = keras_output.argmax(axis=1)
  keras_corrects += np.sum(keras_preds == labels)

  pytorch_model = pytorch_model.cuda()
  pytorch_output = pytorch_model(images.cuda()).cpu().data.numpy()
  pytorch_preds = pytorch_output.argmax(axis=1)
  pytorch_corrects += np.sum(pytorch_preds == labels)

  total_images += len(images)

  if False in np.equal(keras_preds, pytorch_preds): 
    print('KERAS PREDS DIFFERENT FROM TORCH PREDS')
    break

print('==================')
print('KERAS ACC: ', keras_corrects/total_images)
print('PYTORCH ACC: ', pytorch_corrects/total_images)

In [ ]:
generators = preparaDadosKeras(224)
model = get_converted_model()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.evaluate(generators['test'])

# Poda

In [ ]:
models_names = ['Resnet18', 'VGG', 'Densenet121', 'AlexNet', 'GoogLeNet']
input_size = 224
loaders = prepararDados(input_size)
for model_name in models_names:
  model = carrega_model(model_name)
  acc, loss = testePruning(model, loaders['test'])
  print(model_name+ ' - Loss: '+ str(loss)[0:5] + ' Accuracy: '+ str(acc)[0:5] +'%')

In [ ]:
input_size = 224
models_names = [ 'Resnet18', 'VGG',  'Densenet121', 'AlexNet', 'GoogLeNet']
acc_list_models = {'Resnet18': [], 'VGG': [], 'Densenet121': [], 'AlexNet': [], 'GoogLeNet': []}
loss_list_models = {'Resnet18': [], 'VGG': [], 'Densenet121': [], 'AlexNet': [], 'GoogLeNet': []}
params_zerados_models = {'Resnet18': [], 'VGG': [], 'Densenet121': [], 'AlexNet': [], 'GoogLeNet': []}
loaders = prepararDados(input_size)
for model_name in models_names:
  model = carrega_model(model_name) 
  pytorch_total_params = 0
  for layer in model.modules(): 
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
      p = layer.weight.detach().numpy()
      pytorch_total_params += p.size
  std = calcSTD(model_name)
  print('==================== '+model_name+' ======================')
  print('STD: ', std)
  frac = [2*std, std, std/2, std/4, std/6, std/6, std/10, std/12, std/14, std/16, std/18, std/20, std/25, std/30, std/35, std/40, std/45, std/50]

  best_acc = 0.0
  for f in frac: 
    # Load the original unprunned model
    model = carrega_model(model_name)
   
    zeroWeights = 0   
    # For each layer except the output one
    for name, layer in model.named_modules():  
      if isinstance(layer, nn.Conv2d) or (isinstance(layer, nn.Linear) and layer.out_features != 2):
        w = layer.weight.detach().numpy()
        w[abs(w) < f] = 0
        layer.weight = torch.nn.Parameter(torch.tensor(w))
        zeroWeights += np.count_nonzero(w==0)
    
    accuracy, loss = testePruning(model, loaders['test'])
    if accuracy > best_acc:
      torch.save(model.state_dict(), '/content/drive/My Drive/Modelos Salvos/'+model_name+'/'+ model_name.lower() +'-prunned.acc')
      best_acc = accuracy
    params_zerados_models[model_name].append(zeroWeights/pytorch_total_params)
    acc_list_models[model_name].append(accuracy)
    loss_list_models[model_name].append(loss)
    print('===================================================')
    print('Loss: '+ str(loss)[0:6] + ' Accuracy: '+ str(accuracy)[0:6] +'%')
    print('Parâmetros zerados:', zeroWeights)
    print('% em relação ao modelo ', str(zeroWeights/pytorch_total_params)[0:6])

In [ ]:
for model_name in models_names:
  # plotar
  mpl.rcParams['figure.figsize'] = (7, 4) #fig size
  mpl.rcParams['axes.grid'] = False
  plt.plot(params_zerados_models[model_name], acc_list_models[model_name], label=model_name)
  plt.xlabel('Parâmetros zerados (%)')
  plt.ylabel('Accuracy')
  plt.grid(True, which='both', axis='both')  
  plt.tight_layout()
  plt.legend()

In [ ]:
models_names = ['Resnet18', 'VGG', 'Densenet121', 'AlexNet', 'GoogLeNet']
input_size = 224
loaders = prepararDados(input_size)
for model_name in models_names:
  model = carrega_model_prunned(model_name)
  acc, loss = testePruning(model, loaders['test'])
  print(model_name+ ' - Loss: '+ str(loss)[0:5] + ' Accuracy: '+ str(acc)[0:5] +'%')

|    Modelo   | Convencional | % zerada | Pruning |
|:-----------:|:------------:|:--------:|:-------:|
|   Resnet18  |     94.39    |   39.43   |  94.87  |
|     VGG     |     91.35    |   87.35   |  91.35  |
|   AlexNet   |     92.46    |   91.15   |  92.94  |
|  GoogLeNet  |     92.62    |   24.75   |  92.78  |
| Densenet121 |     93.10    |   23.63   |   93.26  |

Poda modelo estudante (destilado) ACC do modelo podado = 90.38 com 4.01% dos parâmetros zerados

In [ ]:
input_size = 224
loaders = prepararDados(input_size)
model = carregar_cnn('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno_kd_v2.acc')
pytorch_total_params = 0
for layer in model.modules(): 
  if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
    p = layer.weight.detach().numpy()
    pytorch_total_params += p.size

std = calcSTD('Aluno')
print('==================== Aluno ======================')
print('STD: ', std)
frac = [2*std, std, std/2, std/4, std/6, std/6, std/10, std/12, std/14, std/16, std/18, std/20, std/25, std/30, std/35, std/40, std/45, std/50]

best_acc = 0.0
for f in frac: 
  # Load the original unprunned model
  model = carrega_model('Aluno')
  
  zeroWeights = 0   
  # For each layer except the output one
  for name, layer in model.named_modules():  
    if isinstance(layer, nn.Conv2d) or (isinstance(layer, nn.Linear) and layer.out_features != 2):
      w = layer.weight.detach().numpy()
      w[abs(w) < f] = 0
      layer.weight = torch.nn.Parameter(torch.tensor(w))
      zeroWeights += np.count_nonzero(w==0)
  
  accuracy, loss = testePruning(model, loaders['test'])
  if accuracy > best_acc:
    torch.save(model.state_dict(), '/content/drive/My Drive/Modelos Salvos/Aluno/'+'aluno-kd-prunned.acc')
    best_acc = accuracy
 
  print('===================================================')
  print('Loss: '+ str(loss)[0:6] + ' Accuracy: '+ str(accuracy)[0:6] +'%')
  print('Parâmetros zerados:', zeroWeights)
  print('% em relação ao modelo ', str(zeroWeights/pytorch_total_params)[0:6])

In [ ]:
input_size = 224
loaders = prepararDados(input_size)
model = carregar_cnn('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned.acc')
print('Modelo estudante destilado e podado - Test set')
metricasModelo(model, loaders['test'])

# Pytorch to Keras Model Conversion

Pytorch to keras conversion

In [ ]:
loaders = prepararDados(224)
models_names = [ 'AlexNet', 'Resnet18', 'GoogLeNet', 'Densenet121', 'VGG' ]
for model_name in models_names: 

  pytorch_model = carrega_model_prunned(model_name)
  keras_model = convert_model(next(iter(loaders['test']))[0].shape, pytorch_model)
  keras_model.save('/content/drive/My Drive/Modelos Salvos/'+model_name+'/'+model_name.lower()+'-prunned-keras.h5')

  keras_corrects = 0
  pytorch_corrects = 0
  total_images = 0

  for batch in tqdm(loaders['test']):
    images, labels =  batch
    labels = labels.data.numpy()

    keras_input = images.data.numpy().transpose(0, 2, 3, 1)
    keras_output = keras_model.predict(keras_input)
    keras_preds = keras_output.argmax(axis=1)
    keras_corrects += np.sum(keras_preds == labels)

    pytorch_model = pytorch_model.cuda()
    pytorch_output = pytorch_model(images.cuda()).cpu().data.numpy()
    pytorch_preds = pytorch_output.argmax(axis=1)
    pytorch_corrects += np.sum(pytorch_preds == labels)

    total_images += len(images)

    if False in np.equal(keras_preds, pytorch_preds): 
      print('KERAS PREDS DIFFERENT FROM TORCH PREDS')
      break

  print('==================')
  print(model_name)
  print('KERAS ACC: ', keras_corrects/total_images)
  print('PYTORCH ACC: ', pytorch_corrects/total_images)

In [ ]:
loaders = prepararDados(224)
models_names = [ 'AlexNet', 'Resnet18']
for model_name in models_names: 

  pytorch_model = carrega_model_prunned(model_name)
  keras_model = carrega_model_prunned_keras(model_name)
  pytorch_model = pytorch_model.eval()
  keras_corrects = 0
  pytorch_corrects = 0
  total_images = 0

  for batch in tqdm(loaders['test']):
    images, labels =  batch
    labels = labels.data.numpy()

    keras_input = images.data.numpy().transpose(0, 2, 3, 1)
    keras_output = keras_model.predict(keras_input)
    keras_preds = keras_output.argmax(axis=1)
    keras_corrects += np.sum(keras_preds == labels)

    pytorch_model = pytorch_model.cuda()
    pytorch_output = pytorch_model(images.cuda()).cpu().data.numpy()
    pytorch_preds = pytorch_output.argmax(axis=1)
    pytorch_corrects += np.sum(pytorch_preds == labels)

    total_images += len(images)

    if False in np.equal(keras_preds, pytorch_preds): 
      print('KERAS PREDS DIFFERENT FROM TORCH PREDS')
      break

  print('==================')
  print(model_name)
  print('KERAS ACC: ', keras_corrects/total_images)
  print('PYTORCH ACC: ', pytorch_corrects/total_images)

# Quantização


Acurácia do modelo antes de quantizar

In [ ]:
generators = preparaDadosKeras(224)
loaders = prepararDados(224)
model = get_converted_model()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.evaluate(generators['test'])

## Quantization aware training

In [ ]:
generators = preparaDadosKeras(224)

model = get_converted_model()

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

q_aware_model.summary()

Acurácia após quantizar

In [ ]:
acc = q_aware_model.evaluate(generators['test'])
print(acc)

Fit do modelo quantizado

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-keras.h5", monitor = 'val_accuracy', mode='max', verbose=1, save_best_only=True)
q_aware_model.fit(generators['train'], validation_data=generators['test'], epochs=25, callbacks=[checkpointer])

In [ ]:
with tfmot.quantization.keras.quantize_scope():
  model = keras.models.load_model('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-keras.h5')
  q_aware_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  
  acc = model.evaluate(generators['test'])
  print(acc)

## Post training quantization

In [ ]:
def metricasModeloQuantizado(y_pred, y_test):
  target_names = ['normal', 'pneumonia']
  print(classification_report(y_test, y_pred, target_names=target_names, digits=4))
  print('\n========= Confusion Matrix ============\n')
  confmat = confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0,1])
  fig, ax = plt.subplots(figsize=(3,3))
  ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
  for i in range(confmat.shape[0]):
      for j in range(confmat.shape[1]):
          ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
  plt.xlabel('predicted label')
  plt.ylabel('true label')
  plt.tight_layout()
  plt.savefig('confusion_matrix.png', dpi=300)
  plt.show()

Convertendo pra tflite

In [ ]:
model = get_converted_model()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite", "wb").write(tflite_model)

TFlite model ACC = 90.38461538461539

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite")

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
corrects = 0
n = 0
k = 0 
for batch in loaders['test']:
    images, labels =  batch
    y = labels.data.numpy()
    x = images.data.numpy()    
    for i in range(len(x)): 
      input_data = np.array(x[i].reshape(input_shape),dtype=np.float32)
      interpreter.allocate_tensors()
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()
      output_data = interpreter.get_tensor(output_details[0]['index'])
      if np.argmax(output_data) == int(y[i]):
        corrects += 1
      n += 1
acc = corrects/n
print('\n ACC =', acc*100)

### Dynamic Range Quantization

In [ ]:
model = get_converted_model()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-dynamic-quantized.tflite", "wb").write(tflite_model)

In [ ]:
print("Float model in KB:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite')/1000)
print("Quantized model in KB:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-dynamic-quantized.tflite')/1000 )
print("Compression ratio:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite')/os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-dynamic-quantized.tflite'))

Post training Dynamic Range Quantization TFlite model  ACC = 90.22435897435898

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-dynamic-quantized.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
corrects = 0
n = 0
tot_predictions = []
tot_labels = []
for batch in loaders['test']:
    images, labels =  batch
    y = labels.data.numpy()
    x = images.data.numpy()
    for i in range(len(x)): 
      input_data = x[i].reshape(input_shape)
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()
      output_data = interpreter.get_tensor(output_details[0]['index'])
      if np.argmax(output_data) == int(y[i]):
        corrects += 1
      n += 1
      tot_predictions.append(np.argmax(output_data))
    tot_labels.extend(y)

acc = corrects/n
print('\n ACC =', acc*100)
metricasModeloQuantizado(tot_predictions, tot_labels)

### Float16 Quantization

In [ ]:
model = get_converted_model()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-fp16.tflite", "wb").write(tflite_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


INFO:tensorflow:Assets written to: /tmp/tmpt_a07xkm/assets


INFO:tensorflow:Assets written to: /tmp/tmpt_a07xkm/assets


67824

In [ ]:
print("Float model in KB:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite')/1000)
print("Quantized model in KB:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-fp16.tflite')/1000 )
print("Compression ratio:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite')/os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-fp16.tflite'))

Post training Float16 Quantization TFlite model ACC = 90.38461538461539

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-fp16.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
corrects = 0
n = 0
tot_predictions = []
tot_labels = []
for batch in loaders['test']:
    images, labels =  batch
    y = labels.data.numpy()
    x = images.data.numpy()
    for i in range(len(x)): 
      input_data = x[i].reshape(input_shape)
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()
      output_data = interpreter.get_tensor(output_details[0]['index'])
      if np.argmax(output_data) == int(y[i]):
        corrects += 1
      n += 1
      tot_predictions.append(np.argmax(output_data))
    tot_labels.extend(y)
acc = corrects/n
print('\n ACC =', acc*100)
metricasModeloQuantizado(tot_predictions, tot_labels)

### Full Integer Quantization

In [ ]:
generators = preparaDadosKeras(224, bs=1)
def representative_data_gen():
  k = 0
  for x,y in generators['train']:
    k += 1
    if k == 300: 
      break
    yield [x]

model = get_converted_model()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-full-integer.tflite", "wb").write(tflite_model)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


INFO:tensorflow:Assets written to: /tmp/tmp1m7kwve6/assets


INFO:tensorflow:Assets written to: /tmp/tmp1m7kwve6/assets


40704

In [ ]:
print("Float model in KB:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite')/1000)
print("Quantized model in KB:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-full-integer.tflite')/1000 )
print("Compression ratio:", os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-keras.tflite')/os.path.getsize('/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-full-integer.tflite'))

Post training Full Integer Quantization TFlite model ACC = 90.86538461538461

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Modelos Salvos/Aluno/aluno-prunned-quantized-full-integer.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
corrects = 0
n = 0
tot_predictions = []
tot_labels = []
for batch in loaders['test']:
    images, labels =  batch
    y = labels.data.numpy()
    x = images.data.numpy()
    for i in range(len(x)): 
      input_data = x[i].reshape(input_shape)
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()
      output_data = interpreter.get_tensor(output_details[0]['index'])
      if np.argmax(output_data) == int(y[i]):
        corrects += 1
      n += 1
      tot_predictions.append(np.argmax(output_data))
    tot_labels.extend(y)
acc = corrects/n
print('\n ACC =', acc*100)
metricasModeloQuantizado(tot_predictions, tot_labels)